# Feature performance comparison

The purpose of this notebook is to compare the classification performance of the individual features, and their combination, for A. Thaliana

# Imports

In [1]:
from sklearnex import patch_sklearn
patch_sklearn()
import os
import sys
from IPython.display import display

sys.path.append('../src')
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA, KernelPCA
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.feature_selection import SelectKBest, RFE, VarianceThreshold
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.base import clone
from scipy.stats import shapiro
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator,TransformerMixin

from yellowbrick.features import ParallelCoordinates
from yellowbrick.features import Rank1D, Rank2D

import pandas as pd
import numpy as np
import seaborn as sns

from dataset.transporter_dataset import create_dataset
from dataset.cluster_fasta import cd_hit
from features.labels import fasta_to_labels
from features.compositions import calculate_composition_feature
from features.pssm import calculate_pssm_feature
from features.coexp import calculate_coexp_feature
from models.eval import nested_crossval
from visualization.feature_plots import create_plot

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


# Globals

In [2]:
ORGANISM="athaliana"
LOG_FILE = f"../logs/{ORGANISM}_amino_sugar.log"
N_THREADS = 16
IDENTITY_THRESHOLD=70
TAX_ID = 3702
OUTLIERS = ["O81775", "Q9SW07", "Q9FHH5", "Q8S8A0", "Q3E965", "Q3EAV6", "Q3E8L0"]

# Dataset

In [3]:
# Delete previous log
if os.path.exists(LOG_FILE):
    with open(LOG_FILE, "w"):
        pass

create_dataset(
    keywords_substrate_filter=["Amino-acid transport", "Sugar transport"],
    keywords_component_filter=["Membrane"],
    keywords_transport_filter=["Transport"],
    input_file="../data/raw/swissprot/uniprot-reviewed_yes.tab.gz",
    multi_substrate="remove",
    outliers=OUTLIERS,
    verbose=True,
    tax_ids_filter=[TAX_ID],
    output_tsv=f"../data/datasets/{ORGANISM}_amino_sugar.tsv",
    output_fasta=f"../data/datasets/{ORGANISM}_amino_sugar.fasta",
    output_log=LOG_FILE,
)

## Clustering

In [4]:
cd_hit(
    executable_location="cd-hit",
    input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar.fasta",
    output_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
    log_file=LOG_FILE,
    identity_threshold=IDENTITY_THRESHOLD,
    n_threads=N_THREADS,
    memory=4096,
    verbose=True,
)

## Annotations

In [5]:
df_annotations = pd.read_table(f"../data/datasets/{ORGANISM}_amino_sugar.tsv", index_col=0)
df_annotations.head()

,keywords_transport,keywords_location,keywords_transport_related,gene_names,protein_names,tcdb_id,organism_id,sequence
Uniprot,,,,,,,,
Q9SFG0,Sugar transport,Membrane;Transmembrane,Symport;Transport,STP6 At3g05960 F2O10.8,Sugar transport protein 6 (Hexose transporter 6),2.A.1.1.56,3702,MAVVVSNANAPAFEAKMTVYVFICVMIAAVGGLIFGYDIGISGGVS...
Q9ZVK6,Sugar transport,Cell membrane;Membrane;Transmembrane,Symport;Transport,SUC8 At2g14670 T6B13.9,Sucrose transport protein SUC8 (Sucrose permea...,NaN,3702,MSDLQAKNDVVAVDRQSSSSLADLDGPSPLRKMISVASIAAGIQFG...
Q84WN3,Sugar transport,Membrane;Transmembrane,Transport,SWEET17 At4g15920 dl4000c FCAALL.237,Bidirectional sugar transporter SWEET17 (AtSWE...,NaN,3702,MAEASFYIGVIGNVISVLVFLSPVETFWKIVKRRSTEEYKSLPYIC...
O04249,Sugar transport,Cell membrane;Membrane;Transmembrane,Symport;Transport,STP7 At4g02050 AGAA.1 T10M13.6,Sugar transport protein 7 (Hexose transporter 7),NaN,3702,MAGGSFGPTGVAKERAEQYQGKVTSYVIIACLVAAIGGSIFGYDIG...
Q56ZZ7,Sugar transport,Membrane;Plastid inner membrane;Transmembrane,Transport,At5g16150 T21H19.70,Plastidic glucose transporter 4 (AtpGlcT),2.A.1.1.102,3702,MQSSTYAVKGNAAFAFQRRTFSSDRSTTSTGIRFAGYKSLATTGPL...


# Feature generation

## Labels

In [6]:
fasta_to_labels(
    input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
    output_tsv=f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_labels.tsv",
)
df_labels = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_labels.tsv",
    index_col=0,
)
df_labels.labels.value_counts()

Sugar transport         84
Amino-acid transport    33
Name: labels, dtype: int64

## AAC, PAAC

In [7]:
for composition_type in ["aac", "paac"]:
    calculate_composition_feature(
        input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
        output_tsv=f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_{composition_type}.tsv",
        feature_type=composition_type,
    )

## PSSM

In [8]:
for uniref_cluster_threshold in [50, 90]:
    for psiblast_iterations in [1, 3]:
        calculate_pssm_feature(
            input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
            output_tsv="../data/features/{}_amino_sugar_cluster{}_pssm_ur{}_{}it.tsv".format(
                ORGANISM, IDENTITY_THRESHOLD, uniref_cluster_threshold, psiblast_iterations
            ),
            tmp_folder="../data/intermediate/blast/pssm_uniref{}_{}it".format(
                uniref_cluster_threshold, psiblast_iterations
            ),
            blast_db="../data/raw/uniref/uniref{}/uniref{}.fasta".format(
                uniref_cluster_threshold, uniref_cluster_threshold
            ),
            iterations=psiblast_iterations,
            psiblast_executable="psiblast",
            psiblast_threads=N_THREADS,
            verbose=False,
        )

## COEXP

In [9]:
# TODO: optimize parameters first

## Reading dataframes

In [10]:

df_aac = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_aac.tsv",
    index_col=0,
)
df_paac = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_paac.tsv",
    index_col=0,
)
df_pssm_50_1it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur50_1it.tsv",
    index_col=0,
)
df_pssm_50_3it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur50_3it.tsv",
    index_col=0,
)
df_pssm_90_1it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur90_1it.tsv",
    index_col=0,
)
df_pssm_90_3it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur90_3it.tsv",
    index_col=0,
)

## Combining dataframes

In [11]:
df_all = pd.concat(
    [
        df_aac,
        df_paac,
        df_pssm_50_1it.rename(columns=lambda c: "PSSM_" + c + "_50_1"),
        df_pssm_50_3it.rename(columns=lambda c: "PSSM_" + c + "_50_3"),
        df_pssm_90_1it.rename(columns=lambda c: "PSSM_" + c + "_90_1"),
        df_pssm_90_3it.rename(columns=lambda c: "PSSM_" + c + "_90_3"),
    ],
    axis=1,
)

df_all

,A,C,D,E,F,G,H,I,K,L,...,PSSM_VL_90_3,PSSM_VK_90_3,PSSM_VM_90_3,PSSM_VF_90_3,PSSM_VP_90_3,PSSM_VS_90_3,PSSM_VT_90_3,PSSM_VW_90_3,PSSM_VY_90_3,PSSM_VV_90_3
Uniprot,,,,,,,,,,,,,,,,,,,,,
Q9SFG0,0.090730,0.015779,0.033531,0.035503,0.086785,0.088757,0.015779,0.092702,0.033531,0.104536,...,0.434307,0.381387,0.421533,0.578467,0.357664,0.390511,0.392336,0.512774,0.656934,0.417883
Q84WN3,0.066390,0.004149,0.020747,0.053942,0.070539,0.074689,0.008299,0.070539,0.041494,0.120332,...,0.383260,0.264317,0.374449,0.726872,0.215859,0.286344,0.312775,0.493392,0.982379,0.352423
O04249,0.079922,0.013645,0.017544,0.042885,0.066277,0.107212,0.011696,0.076023,0.035088,0.120858,...,0.476898,0.415842,0.471947,0.592409,0.387789,0.415842,0.422442,0.514851,0.702970,0.450495
Q56ZZ7,0.128205,0.009158,0.020147,0.038462,0.051282,0.106227,0.003663,0.058608,0.031136,0.120879,...,0.488636,0.433442,0.472403,0.608766,0.420455,0.461039,0.462662,0.514610,0.657468,0.470779
Q8H184,0.065217,0.012077,0.045894,0.057971,0.084541,0.062802,0.009662,0.050725,0.050725,0.111111,...,0.486772,0.391534,0.473545,0.595238,0.370370,0.417989,0.431217,0.505291,0.679894,0.473545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q94B65,0.095975,0.009288,0.015480,0.027864,0.055728,0.077399,0.012384,0.068111,0.055728,0.148607,...,0.542056,0.489097,0.529595,0.716511,0.426791,0.523364,0.504673,0.613707,0.894081,0.520249
Q0WWW9,0.082437,0.010753,0.028674,0.037634,0.066308,0.114695,0.007168,0.060932,0.039427,0.154122,...,0.544582,0.465021,0.515775,0.632373,0.455418,0.486968,0.482853,0.539095,0.703704,0.514403
Q2V4B9,0.092929,0.020202,0.024242,0.038384,0.062626,0.105051,0.008081,0.050505,0.036364,0.137374,...,0.521151,0.495770,0.510998,0.566836,0.477157,0.502538,0.502538,0.509306,0.602369,0.500846


## Custom Transformer to try all parameters

New version: Ignores all additional features that don't start with "PSSM"

In [12]:
class PSSMSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_names, uniref_threshold="all", iterations="all"):
        self.feature_names = feature_names
        self.uniref_threshold = uniref_threshold
        self.iterations = iterations

    def fit(self, X, y=None):
        if self.uniref_threshold in {50, 90}:
            has_uniref = (
                np.char.find(self.feature_names, str(self.uniref_threshold)) >= 0
            )
        elif self.uniref_threshold == "all":
            has_uniref = np.array([True] * len(self.feature_names))
        else:
            raise ValueError(f"Incorrect uniref threshold {self.uniref_threshold}")

        if self.iterations in {1, 3}:
            has_iterations = np.char.find(self.feature_names, str(self.iterations)) >= 0
        elif self.iterations == "all":
            has_iterations = np.array([True] * len(self.feature_names))
        else:
            raise ValueError(f"Incorrect iteration count: {self.iterations}")
        
        not_pssm_feature = ~np.char.startswith(self.feature_names, "PSSM")
        self.mask = np.bitwise_or(np.bitwise_and(has_uniref, has_iterations), not_pssm_feature)
        return self

    def transform(self, X, y=None):
        X = np.array(X)
        X = X[:, self.mask]
        return X


# Functions

In [13]:
def get_feature_stats(df_features, df_labels_, labels=["Amino-acid transport", "Sugar transport"]):
    df_stats = pd.concat(
        {
            "corr": df_features.corrwith(
                df_labels_.labels.transform(lambda x: 1.0 if x == labels[1] else 0.0)
            ),
            "mean": df_features.mean(),
            "std": df_features.std(),
        },
        axis=1,
    )

    df_stats["sum"] = df_stats.sum(axis=1)
    df_stats["corr_abs"] = df_stats["corr"].abs()

    df_stats["mean0"] = df_features.loc[df_labels_[df_labels_.labels == labels[0]].index].mean()
    df_stats["mean1"] = df_features.loc[df_labels_[df_labels_.labels == labels[1]].index].mean()

    df_stats["median0"] = df_features.loc[
        df_labels_[df_labels_.labels == labels[0]].index
    ].median()
    df_stats["median1"] = df_features.loc[
        df_labels_[df_labels_.labels == labels[1]].index
    ].median()

    df_stats["mediandiff"] = (df_stats["median0"] - df_stats["median1"]).abs()
    df_stats = df_stats.sort_values("mediandiff", ascending=False)
    return df_stats

In [14]:
def get_independent_test_set(
    df_features, df_labels_, labels=["Amino-acid transport", "Sugar transport"], test_size=0.2
):
    X = df_features.to_numpy()
    y = np.where(df_labels_.labels == labels[1], 1, 0)
    feature_names = df_features.columns.to_numpy(dtype=str)
    sample_names = df_features.index.to_numpy(dtype=str)
    (
        X_train,
        X_test,
        y_train,
        y_test,
        sample_names_train,
        sample_names_test,
    ) = train_test_split(
        X, y, sample_names, stratify=y, random_state=42, shuffle=True, test_size=test_size
    )
    return (
        X_train,
        X_test,
        y_train,
        y_test,
        sample_names_train,
        sample_names_test,
        feature_names,
    )


In [15]:
def print_validation_results(y_true_, y_pred_, labels = ["Amino", "Sugar"]):
    report_dict = classification_report(y_true=y_true_, y_pred=y_pred_, output_dict=True)
    report_dict = {
        labels[0]: report_dict['0'],
        labels[1]: report_dict['1'],
        "Macro": report_dict["macro avg"],
        "Weighted": report_dict["weighted avg"]
    }
    report_df = pd.DataFrame.from_dict(report_dict)
    confusion_matrix_df = pd.DataFrame(
        confusion_matrix(y_true_, y_pred_),
        columns=labels,
        index=labels,
    )
    return report_df, confusion_matrix_df

# Combined Features

### Stats, Plots

Only three of the top 30 features comes from PAAC, all others come from PSSM with Uniref50 and 3 iterations. This is also the best parameter PSSM dataset that was found in the PSSM notebook.

In [16]:
df_stats = get_feature_stats(df_all, df_labels)
df_stats.sort_values("corr_abs", ascending=False).head(30)

,corr,mean,std,sum,corr_abs,mean0,mean1,median0,median1,mediandiff
PSSM_HH_50_3,-0.577959,0.583338,0.047090,0.052469,0.577959,0.626574,0.566352,0.629730,0.566020,0.063709
PSSM_HC_50_3,-0.529869,0.453508,0.071213,-0.005147,0.529869,0.513452,0.429959,0.522777,0.441588,0.081189
CY,-0.527530,0.000768,0.001546,-0.525216,0.527530,0.002063,0.000259,0.001905,0.000000,0.001905
PSSM_HY_50_3,-0.513045,0.500542,0.066845,0.054341,0.513045,0.555022,0.479138,0.552910,0.489270,0.063640
PSSM_HW_50_3,-0.504588,0.482668,0.077551,0.055631,0.504588,0.544832,0.458246,0.548807,0.462431,0.086376
PSSM_RC_50_3,-0.468761,0.666274,0.058030,0.255543,0.468761,0.709488,0.649297,0.708333,0.650152,0.058181
PSSM_TC_50_3,-0.464902,0.528368,0.055631,0.119098,0.464902,0.569454,0.512227,0.564159,0.512447,0.051712
PSSM_II_50_3,-0.455575,0.794808,0.066247,0.405480,0.455575,0.842753,0.775973,0.843648,0.778167,0.065481
PSSM_WI_50_3,-0.452175,0.793433,0.067062,0.408320,0.452175,0.841606,0.774508,0.829630,0.765606,0.064024
PSSM_HF_50_3,-0.451090,0.466087,0.076532,0.091529,0.451090,0.520931,0.444542,0.521277,0.456896,0.064380


### Independent test set

In [17]:
(
    X_train,
    X_test,
    y_train,
    y_test,
    sample_names_train,
    sample_names_test,
    feature_names,
) = get_independent_test_set(df_all, df_labels, test_size=0.2)

### Model selection

Linear SVC could be a good choice to avoid overfitting.

In [18]:
for estimator in [
    LinearSVC(max_iter=1e6, class_weight="balanced"),
    SVC(class_weight="balanced"),
    RandomForestClassifier(class_weight="balanced"),
    LinearSVC(max_iter=1e6),
    SVC(),
    RandomForestClassifier(),
    GaussianNB(),
    KNeighborsClassifier(),
    SGDClassifier(),
]:
    pipe = make_pipeline(StandardScaler(), estimator)
    scores = cross_val_score(pipe, X_train, y_train, scoring="f1_macro")
    print("### ", str(estimator))
    print(f"CV folds: {scores.round(3)}")
    print(f"Mean: {scores.mean().round(3)}")
    print(f"Std: {scores.std().round(3)}")


###  LinearSVC(class_weight='balanced', max_iter=1000000.0)
CV folds: [0.878 1.    0.942 0.935 0.819]
Mean: 0.915
Std: 0.061
###  SVC(class_weight='balanced')
CV folds: [0.737 0.737 0.936 1.    0.734]
Mean: 0.829
Std: 0.115
###  RandomForestClassifier(class_weight='balanced')
CV folds: [0.864 0.864 0.887 1.    0.875]
Mean: 0.898
Std: 0.052
###  LinearSVC(max_iter=1000000.0)
CV folds: [0.878 1.    0.942 0.935 0.819]
Mean: 0.915
Std: 0.061
###  SVC()
CV folds: [0.737 0.737 0.936 1.    0.734]
Mean: 0.829
Std: 0.115
###  RandomForestClassifier()
CV folds: [0.604 0.842 1.    0.839 0.839]
Mean: 0.825
Std: 0.127
###  GaussianNB()
CV folds: [0.729 0.842 0.878 0.778 0.557]
Mean: 0.757
Std: 0.112
###  KNeighborsClassifier()
CV folds: [0.808 0.683 0.878 0.926 0.734]
Mean: 0.806
Std: 0.089
###  SGDClassifier()
CV folds: [0.825 0.936 0.942 0.935 0.819]
Mean: 0.891
Std: 0.057


### Parameter tuning

In [19]:
gsearch = GridSearchCV(
    estimator=make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        LinearSVC(max_iter=1e6),
    ),
    param_grid={
        "pssmselector__uniref_threshold": [50, 90, "all"],
        "pssmselector__iterations": [1, 3, "all"],
        "linearsvc__class_weight": ["balanced"],
        "linearsvc__C": [0.1, 1, 10],
        "linearsvc__dual": [True, False],
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_svc = gsearch.best_estimator_


{'linearsvc__C': 10, 'linearsvc__class_weight': 'balanced', 'linearsvc__dual': False, 'pssmselector__iterations': 3, 'pssmselector__uniref_threshold': 50}
0.9622053872053872


### Dimensionality reduction

In [20]:
pca = PCA()
pca.fit(X_train)
csum = np.cumsum(pca.explained_variance_ratio_)
print("Number of components to explain 97% of variance:", np.argmax(csum >= 0.97) + 1)

Number of components to explain 97% of variance: 46


In [21]:
gsearch = GridSearchCV(
    estimator=make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        PCA(),
        StandardScaler(),
        LinearSVC(max_iter=1e6),
    ),
    param_grid={
        "pssmselector__uniref_threshold": [50, 90, "all"],
        "pssmselector__iterations": [1, 3, "all"],
        "linearsvc__class_weight": ["balanced"],
        "linearsvc__C": [0.1, 1, 10],
        "linearsvc__dual": [True, False],
        "pca__n_components": np.linspace(0.8, 0.99, 20)
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_svc_pca = gsearch.best_estimator_

{'linearsvc__C': 0.1, 'linearsvc__class_weight': 'balanced', 'linearsvc__dual': True, 'pca__n_components': 0.97, 'pssmselector__iterations': 3, 'pssmselector__uniref_threshold': 'all'}
1.0


In [22]:
gsearch = GridSearchCV(
    estimator=make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        SelectKBest(),
        StandardScaler(),
        LinearSVC(max_iter=1e6),
    ),
    param_grid={
        "pssmselector__uniref_threshold": [50, 90, "all"],
        "pssmselector__iterations": [1, 3, "all"],
        "linearsvc__class_weight": ["balanced"],
        "linearsvc__C": [0.1, 1, 10],
        "linearsvc__dual": [True, False],
        "selectkbest__k": list(range(1,400))
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_svc_kbest = gsearch.best_estimator_

{'linearsvc__C': 10, 'linearsvc__class_weight': 'balanced', 'linearsvc__dual': False, 'pssmselector__iterations': 3, 'pssmselector__uniref_threshold': 'all', 'selectkbest__k': 286}
0.9872053872053872


### Validation

In [23]:
best_estimator = best_estimator_svc_pca
best_scores = cross_val_score(
    estimator=clone(best_estimator), X=X_train, y=y_train, scoring="f1_macro"
)
print(f"Train scores: {best_scores.mean().round(3)}+-{best_scores.std().round(3)}")

y_pred = best_estimator.predict(X_test)
y_true = y_test.copy()

report_df, confusion_matrix_df = print_validation_results(y_true, y_pred, labels=["Amino", "Sugar"])
display(report_df.round(3))
display(confusion_matrix_df)

Train scores: 1.0+-0.0


,Amino,Sugar,Macro,Weighted
precision,1.0,1.0,1.0,1.0
recall,1.0,1.0,1.0,1.0
f1-score,1.0,1.0,1.0,1.0
support,7.0,17.0,24.0,24.0


,Amino,Sugar
Amino,7,0
Sugar,0,17


In [24]:
best_estimator = best_estimator_svc_kbest
best_scores = cross_val_score(
    estimator=clone(best_estimator), X=X_train, y=y_train, scoring="f1_macro"
)
print(f"Train scores: {best_scores.mean().round(3)}+-{best_scores.std().round(3)}")

y_pred = best_estimator.predict(X_test)
y_true = y_test.copy()

report_df, confusion_matrix_df = print_validation_results(y_true, y_pred, labels=["Amino", "Sugar"])
display(report_df.round(3))
display(confusion_matrix_df)

Train scores: 0.987+-0.026


,Amino,Sugar,Macro,Weighted
precision,1.0,1.0,1.0,1.0
recall,1.0,1.0,1.0,1.0
f1-score,1.0,1.0,1.0,1.0
support,7.0,17.0,24.0,24.0


,Amino,Sugar
Amino,7,0
Sugar,0,17
